In [1]:
import shutil

In [2]:
from experiment.scenario import *
from experiment_setup.exp_utrecht_10_492_594_v2_setup import *

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


## Setup

Set up experiment:

In [3]:
load_existing_experiment = True

if not load_existing_experiment:
    e = Experiment(experiment_name, experiment_dirpath, default_config, scenario_settings, scene_parts)

    print("\nSetting up experiment:")
    e.setup()
else:
    print("\nLoading existing experiment:")
    e = Experiment.load(experiment_dirpath / experiment_name, load_scenarios=True)


Loading existing experiment:

Loading experiment configuration ...
Initializing experiment ...
Loading scenarios ...
- 143 scenarios


## Load experiment & get summary stats

### Load all

In [4]:
# Load optimization experiments

# eos = [
#     Experiment.load(r"C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_055"),
#     Experiment.load(r"C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_055_penalize_complexity"),
#     Experiment.load(r"C:\Users\Florian\Data\city-to-scan-to-city\utrecht_10-492-594_v2\06_reconstruction_optimization\scenario_055_penalize_complexity_harder")
# ]

# Load optimization experiments for all scenarios
scenarios = list(range(121))

eos = []
for si in scenarios:
    print(f"\n{si}", end="")
    eos.append(Experiment.load(e[si].recon_optim_output_dirpath, load_scenarios=True))


0
Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios

1
Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios

2
Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios

3
Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios

4
Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios

5
Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenarios ...
- 51 scenarios

6
Loading experiment configuration ...
File `scenario_settings.json` not found.
Initializing experiment ...
Loading scenari

For any and all optimization scenarios that crashed, move their folders from the settings, reconstruction, and evaluation dirpaths of the optimization experiment to the graveyard directory. Remember to reload the optimization experiments afterwards, otherwise the ghosts of the crashed scenarios are still loaded.

In [ ]:
def remove_unsuccessful_optim_scenarios(damage_str_snippet: str = "crash", dest_dirname: str = "graveyard"):
    dest_dirpath = e.recon_optim_dirpath / dest_dirname
    dest_dirpath.mkdir(exist_ok=True)
    for eo in eos:
        print(f"\nOptim experiment for {eo.name}")
        dirpaths = [eo.settings_dirpath, eo.reconstruction_dirpath, eo.evaluation_dirpath]
        for p in dirpaths:
            for f in p.iterdir():
                if f.is_dir() and damage_str_snippet in f.name:
                    name_new = f"{eo.name}_{p.name}_{f.name}"
                    print(f"- Moving `{f.name}` to `{name_new}`")
                    f = f.rename(f.parent / name_new)
                    shutil.move(f, dest_dirpath)

remove_unsuccessful_optim_scenarios("crash", "graveyard")
remove_unsuccessful_optim_scenarios("broken", "hospital")

In [ ]:
# Number of optimization scenarios in which Geoflow timed out for each optim experiment
for eo in eos:
    n_opt_sc = len(eo)
    n_timeout = len([s for s in eo.scenarios.values() if s.flag_no_recon_output])
    print(f"{eo.name}: {n_timeout} / {n_opt_sc}")

In [ ]:
# Number of optimization scenarios in which zero buildings were reconstructed OR Geoflow timed out for each optim experiment
for eo in eos:
    n_opt_sc = len(eo)
    n_zero_buildings = len([s for s in eo.scenarios.values() if s.flag_zero_buildings_reconstructed])
    print(f"{eo.name}: {n_zero_buildings} / {n_opt_sc}")

In [ ]:
# Update older optimization scenarios with the path to the input geopackage building models file
for eo in eos:
    eo.default_config["evaluation_config"]["input_geopackage_filepath"] = str(input_dirpath / "GPKG" / "10-492-594.gpkg")
    for name, scenario in eo.scenarios.items():
        scenario.config["evaluation_config"]["input_geopackage_filepath"] = str(input_dirpath / "GPKG" / "10-492-594.gpkg")
    eo.save()

In [19]:
for eo in eos:
    eo.run_steps(Scenario.setup_evaluation, lods=["2.2"])


Running 'setup_evaluation' for optim_0000 ...

Finished 'setup_evaluation' for optim_0000 after 0:00:00.021997.

Running 'setup_evaluation' for optim_0001 ...

Finished 'setup_evaluation' for optim_0001 after 0:00:00.015996.

Running 'setup_evaluation' for optim_0002 ...

Finished 'setup_evaluation' for optim_0002 after 0:00:00.017020.

Running 'setup_evaluation' for optim_0003 ...

Finished 'setup_evaluation' for optim_0003 after 0:00:00.015983.

Running 'setup_evaluation' for optim_0004 ...

Finished 'setup_evaluation' for optim_0004 after 0:00:00.014009.

Running 'setup_evaluation' for optim_0005 ...

Finished 'setup_evaluation' for optim_0005 after 0:00:00.015517.

Running 'setup_evaluation' for optim_0006 ...

Finished 'setup_evaluation' for optim_0006 after 0:00:00.013997.

Running 'setup_evaluation' for optim_0007 ...

Finished 'setup_evaluation' for optim_0007 after 0:00:00.018010.

Running 'setup_evaluation' for optim_0008 ...

Finished 'setup_evaluation' for optim_0008 after

In [20]:
# Run additional evaluator to check their results. They may already have been run while running ReconstructionOptimization.select_optimal_scenario().
for eo in eos:
    eo.run_steps(Scenario.run_evaluation, evaluator_selection=["complexity", "complexity_diff", "geoflow_output"])


Running 'run_evaluation' for optim_0000 ...


Finished 'run_evaluation' for optim_0000 after 0:00:00.

Running 'run_evaluation' for optim_0001 ...


Finished 'run_evaluation' for optim_0001 after 0:00:00.

Running 'run_evaluation' for optim_0002 ...


Finished 'run_evaluation' for optim_0002 after 0:00:00.

Running 'run_evaluation' for optim_0003 ...


Finished 'run_evaluation' for optim_0003 after 0:00:00.

Running 'run_evaluation' for optim_0004 ...


Finished 'run_evaluation' for optim_0004 after 0:00:00.

Running 'run_evaluation' for optim_0005 ...

Starting ComplexityEvaluator ...

Counting faces for LOD 2.2 ...

Finished ComplexityEvaluator after 0:00:02.675895.

Starting ComplexityDifferenceEvaluator ...

Starting ComplexityEvaluator ...

Counting faces for LOD 2.2 ...

Finished ComplexityEvaluator after 0:00:01.083143.

Evaluation results for complexity evaluator 2 already exist. Will not reevaluate.

Finished ComplexityDifferenceEvaluator after 0:00:01.098141.

Starting Geofl

### Summary stats

In [21]:
# Compute summary statistics for the evaluators whose results are available
for eo in eos:
    # eo.compute_summary_statistics(evaluator_selection=["hausdorff", "complexity", "iou_3d", "geoflow_output"], ignore_missing=True)
    eo.compute_summary_statistics(evaluator_selection=["hausdorff", "complexity", "complexity_diff", "geoflow_output"], ignore_missing=True)


Computing summary statistics of 51 scenarios in experiment scenario_110...



Computing summary statistics of 51 scenarios in experiment scenario_111...

Computing summary statistics of 51 scenarios in experiment scenario_112...

Computing summary statistics of 51 scenarios in experiment scenario_113...






Computing summary statistics of 51 scenarios in experiment scenario_114...






Computing summary statistics of 51 scenarios in experiment scenario_115...











Computing summary statistics of 51 scenarios in experiment scenario_116...































Computing summary statistics of 51 scenarios in experiment scenario_117...
























Computing summary statistics of 51 scenarios in experiment scenario_118...






Missing output for evaluator `hausdorff` in scenario `optim_0011` ignored.














Computing summary statistics of 51 scenarios in experiment scenario_119...
















Computing summary statistics of 51 scenarios in experim

In [5]:
# If summary stats were computed previously, this will simply load the values
ss = [eo.summary_stats for eo in eos]

Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistics from file 'summary_statistics.csv'...
Loading summary statistic